In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from pinecone import Pinecone,ServerlessSpec
from langchain_pinecone import PineconeVectorStore
import os
import cassio

load_dotenv()


True

In [35]:
file_path = r"C:\Users\iamda\OneDrive\Desktop\Gen ai\LLMops-Project\data\Medical_book.pdf"

def loader_function(file_path):
    pdf = PyPDFLoader(file_path=file_path)
    loader = pdf.load()
    return loader


In [36]:
loader = loader_function(file_path)

def recursive_function(loader):
    spliter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100)
    docs = spliter.split_documents(loader)
    return docs


In [37]:
docs = recursive_function(loader)

In [38]:
len(docs)

3156

In [39]:
def embedding_function():
    hf_embedding = HuggingFaceEmbeddings(model="all-MiniLM-L6-v2")
    return hf_embedding

hf_embedding = embedding_function()

In [57]:
import os

token 
End_point 

In [45]:
cassio.init(
    token=token,
    database_id=End_point
)

In [48]:
from langchain.vectorstores.cassandra import Cassandra

vectore_store_db = Cassandra(embedding=hf_embedding,
          session=None,
          keyspace=None,
          table_name="medical_rag"
          )

from langchain.indexes.vectorstore import VectorStoreIndexWrapper

vectore_store_db.add_documents(docs)

vectore_store_index = VectorStoreIndexWrapper(vectorstore=vectore_store_db)
vectore_store_index

VectorStoreIndexWrapper(vectorstore=<langchain_community.vectorstores.cassandra.Cassandra object at 0x0000018A37593F70>)

In [49]:
retriever = vectore_store_db.as_retriever()


In [50]:
retriever.invoke("i am suffering the high fever so tell me how to overcome")

[Document(id='bdbfcfff75344d499cc7e38a4dbbda8b', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': '60.0', 'page_label': '61', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'C:\\Users\\iamda\\OneDrive\\Desktop\\Gen ai\\LLMops-Project\\data\\Medical_book.pdf', 'text': 'occur. The bacterial infection causes a fever of 100-104°F\n(38°–40°C). In addition, a general ill feeling, muscle\naches, headache, chills, and loss of appetite may be felt.\nDiagnosis\nIf lymphangitis is suspected, the person should call\nhis or her doctor immediately or go to an emergency\nroom. Acute lymphangitis could be diagnosed by the\nfamily doctor, infectious disease specialist, or an emer-\ngency room doctor. The painful, red streaks just below\nthe skin surface and the high fever are diagnostic of acute\nlymphangitis. A sample of blood would be taken for cul-\nture to determine whether the bacteria have entered the\nbloodstream. A bio

In [51]:
prompt = ChatPromptTemplate.from_template( """

    You are a helpfull ai assistant and your work to give the answer of user question 
    based on the context is provided: {context}.
    Make sure question/answer is meet the context if question is not meet the context so
    return Sorry i don't give the answer.
    Question:{input} 

    """
)

In [52]:
llm = ChatGroq(model="llama-3.1-8b-instant")

create_stuff = create_stuff_documents_chain(llm=llm,prompt=prompt)
chain = create_retrieval_chain(retriever,create_stuff)

chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Cassandra', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.cassandra.Cassandra object at 0x0000018A37593F70>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template="\n\n    You are a helpfull ai assistant and your work to give the answer of user question \n    based on the conte

In [55]:
response = chain.invoke({"input":"i am suffering the high fever so tell me how to overcome"})
print(response["answer"])

Based on the context provided, it seems like you are looking for advice on how to overcome a high fever. 

According to the text, the bacterial infection (lymphangitis) causes a fever of 100-104°F (38°–40°C). To overcome a high fever, you can try the following:

1. **Take medication**: Aspirin is recommended to relieve pain and reduce fever. The recommended dosage is one to two tablets every three to four hours, up to six times a day.
2. **Stay hydrated**: Drink plenty of fluids to help your body regulate its temperature and replace lost fluids.
3. **Rest**: Get plenty of rest to help your body fight off the infection.
4. **Seek medical attention**: If you suspect you have lymphangitis, you should seek medical attention immediately.
